<a href="https://colab.research.google.com/github/manrodri/01-02-service-containers/blob/main/Titanic_kaggle_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Titanic - Machine Learning from Disaster

https://www.kaggle.com/c/titanic

Exercise to cement learning of basic NN clasification problems.

**Goal**: The competition is simple: use machine learning to create a model that predicts which passengers survived the Titanic shipwreck.


1. Understand the problem
2. EDA (Exploratory Data Analysis)
3. Train, Tune, Ensenble ML models
4. Make predictions
5. Evaluate the models
6. Upload the model (optional)

## 1. Understanding the problem



On April 15, 1912, during her maiden voyage, the widely considered “unsinkable” RMS Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew.

While there was some element of luck involved in surviving, it seems some groups of people were more likely to survive than others.

**Goal**: Build a predictive model that answers the question: “**what sorts of people were more likely to survive?**” using passenger data (ie name, age, gender, socio-economic class, etc).


This is a binary classification problem: 
- We'll have a number of features in our train data and one dependent variable (survided or perished)


**Submission file format:**
- CSV file with exactly 418 entries plus a header row
- Two columns:
  - PassengerId (sorted in any order)
  - Survived: (contains your binary predictions: 1 for survived, 0 for deceased)

## 2. Exploratory Data Analysis (EDA)

https://www.kaggle.com/competitions/titanic/data

In [ ]:
# connect Google Drive to the Notebook

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import TensorFlow 2.0
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


print(f"TF version: {tf.__version__}")
print(f"TF Hub version: {hub.__version__}")

# Check for GPU availability
print('GPU', 'available (YES!!!!)' if tf.config.list_physical_devices('GPU') else "not available :(" )


TF version: 2.11.0
TF Hub version: 0.12.0
GPU not available :(


In [ ]:
train_csv = pd.read_csv("/content/drive/MyDrive/ML/Kaggle_competitions/Titanic/data/train.csv")
test_csv = pd.read_csv("/content/drive/MyDrive/ML/Kaggle_competitions/Titanic/data/test.csv")

In [ ]:
train_csv.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
train_csv.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_csv.shape

(891, 12)

In [ ]:
train_csv.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

### Data Dictionary
I need to understand what each attribute means. Luckily, Kaggels provides a table explaining this.


| **Variable**  |  **Definition** | 	**Key**   |   
|---|---|---|
|  survival | Survival  |  	0 = No, 1 = Yes |   
|  pclass | Ticket class  |  	1 = 1st, 2 = 2nd, 3 = 3rd |      
| sex  |  Sex |   |  
| sibsp | # of siblings / spouses aboard the Titanic |  |
| parch |  # of parents / children aboard the Titanic	|   |
| ticket | Ticket number |  
| fare | Passenger fare |   |
|cabin |	Cabin number	|   |
| embarked| Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton  |